<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4c5e3ea0ff6fc2aef3599f8dbf1f877638d8eb458009d053106a589798735a43
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 14:55:17
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.41 C
-------------------
Today PnL: -94.73 K (-0.67%)
Current PnL: -24.32 L (-15.94%)
CY Booked + Current PnL: -10.43 L (-6.84%)
-------------------
Total profit:  1.47 L
Total loss:  -25.79 L
-------------------
Total Booked + Current PnL: 16.55 L (13.17%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.82%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.73 L (63.43%)
Deployed:  1.26 C
Current:  1.41 C
CAGR/XIRR %: 6.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.27,-8.90,9.79,0.02,15569.0,-15529.0,159030.0,142.75,66.0,M-SC,14.25,234.0,-1.00,1.12,38.33,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.16,-16.68,20.13,0.09,16902.0,-16813.0,83964.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.59,10.31,OX40N,NTT,DURABLES
51,MASFIN,398.61,0.47,-2.10,24.66,22.05,23655.0,-2055.0,95925.0,-15.54,60.0,H-SC,6.74,164.0,-0.09,0.68,39.43,XR,ATH,FINANCE
43,ITC,452.00,-0.17,-2.21,12.93,10.44,25307.0,-4416.0,195722.0,-41.56,40.0,X-LC,1.52,5.0,-0.17,1.38,3.23,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.89,5.05,14.42,20.19,26405.0,8797.0,183117.0,-17.67,44.0,X-MC,6.71,68.0,0.33,1.29,20.92,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,4.56,-46.97,218.79,69.05,154387.0,-62502.0,70564.0,-57.12,52.0,H-SC,23.80,140.0,-0.40,0.50,8.82,SR,ATH,IT
13,BSOFT,831.70,4.10,-23.57,97.30,50.79,104593.0,-33158.0,107495.0,-1.82,71.0,H-SC,5.89,151.0,-0.32,0.76,25.16,XR,ATH,IT
48,KPIGREEN,730.66,2.99,-10.95,65.03,46.95,72556.0,-13724.0,111573.0,-17.73,39.0,H-SC,2.38,154.0,-0.19,0.79,39.77,MH,ATH,POWER
85,WIPRO,420.00,1.83,4.49,64.87,72.27,114853.0,7603.0,177051.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.25,10.94,XR,NTT,IT
16,CERA,9475.00,1.48,-28.22,72.60,23.89,91663.0,-49645.0,126258.0,-30.79,34.0,H-SC,7.27,157.0,-0.54,0.89,9.04,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-4.96,-39.25,85.84,12.90,113949.0,-85750.0,132746.0,95.91,46.0,H-SC,12.17,161.0,-0.75,0.94,10.38,OX40N,NTT,FINANCE
22,DIXON,18931.72,-3.28,-9.97,35.04,21.58,44210.0,-13968.0,126171.0,-54.61,28.0,X-MC,5.88,56.0,-0.32,0.89,11.70,X40N,ATH,IT
84,WHIRLPOOL,2270.00,-3.28,-21.72,132.39,81.92,118974.0,-24932.0,89866.0,-54.14,22.0,M-SC,4.98,236.0,-0.21,0.64,6.39,XR,NTT,DURABLES
68,SONACOMS,806.63,-3.08,-15.09,64.35,39.54,55270.0,-15269.0,85890.0,-33.56,49.0,M-SC,6.41,220.0,-0.28,0.61,21.11,AR,ATH,AUTO
18,COFFEEDAY,80.00,-2.63,-42.44,135.02,35.27,88245.0,-48192.0,65357.0,-55.73,27.0,L-SC,14.78,270.0,-0.55,0.46,59.96,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.26,-0.3,107.35,106.72,131998.0,-376.0,122960.0,-50.63,38.0,H-SC,1.70,139.0,-0.00,0.87,23.00,AR,ATH,MISC
37,IEX,219.00,-0.01,0.8,47.44,48.63,95618.0,1601.0,201555.0,-32.92,66.0,H-SC,14.79,136.0,0.02,1.42,12.45,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.30,-5.95,121.05,107.91,168097.0,-8778.0,138866.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.27,-8.90,9.79,0.02,15569.0,-15529.0,159030.0,142.75,66.0,M-SC,14.25,234.0,-1.00,1.12,38.33,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.16,-16.68,20.13,0.09,16902.0,-16813.0,83964.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.59,10.31,OX40N,NTT,DURABLES
47,KANSAINER,340.00,1.02,-22.24,45.92,13.47,96294.0,-59967.0,209700.0,-68.12,37.0,H-SC,2.12,158.0,-0.62,1.48,6.63,XY24,NTT,PAINTS
67,SIS,528.00,1.01,-24.52,60.66,21.26,50837.0,-27226.0,83806.0,1976.68,46.0,H-SC,3.91,166.0,-0.54,0.59,13.23,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.83,4.49,64.87,72.27,114853.0,7603.0,177051.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.25,10.94,XR,NTT,IT
37,IEX,219.0,-0.01,0.80,47.44,48.63,95618.0,1601.0,201555.0,-32.92,66.0,H-SC,14.79,136.0,0.02,1.42,12.45,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.83,4.49,64.87,72.27,114853.0,7603.0,177051.0,-10.16,67.0,M-LC,3.13,99.0,0.07,1.25,10.94,XR,NTT,IT
37,IEX,219.00,-0.01,0.80,47.44,48.63,95618.0,1601.0,201555.0,-32.92,66.0,H-SC,14.79,136.0,0.02,1.42,12.45,XR,NTT,MISC
38,INDIAMART,4810.62,-0.26,-0.30,107.35,106.72,131998.0,-376.0,122960.0,-50.63,38.0,H-SC,1.70,139.0,-0.00,0.87,23.00,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-1.07,-1.66,37.91,35.63,78134.0,-3469.0,206105.0,-15.41,41.0,H-MC,3.64,119.0,-0.04,1.46,14.74,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.30,-5.95,121.05,107.91,168097.0,-8778.0,138866.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.23,-20.46,86.65,48.45,138184.0,-41032.0,159474.0,-26.10,14.0,X-MC,8.84,64.0,-0.30,1.13,0.23,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.95,-37.18,115.22,35.20,93137.0,-47836.0,80834.0,5.99,24.0,X-SC,14.21,92.0,-0.51,0.57,0.95,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-1.60,-9.60,39.45,26.07,58394.0,-15720.0,148020.0,-33.39,27.0,X-MC,10.52,55.0,-0.27,1.05,0.00,X40,ATH,APPARELS
8,AWL,485.00,-0.56,-21.51,95.56,53.50,226324.0,-64911.0,236840.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.20,XY24,NTT,FMCG
22,DIXON,18931.72,-3.28,-9.97,35.04,21.58,44210.0,-13968.0,126171.0,-54.61,28.0,X-MC,5.88,56.0,-0.32,0.89,11.70,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.66,-22.69,112.33,64.15,206642.0,-53991.0,183960.0,-54.57,46.0,X-SC,1.45,82.0,-0.26,1.30,3.19,XY24,BTT,CEMENT
43,ITC,452.0,-0.17,-2.21,12.93,10.44,25307.0,-4416.0,195722.0,-41.56,40.0,X-LC,1.52,5.0,-0.17,1.38,3.23,X40,NTT,FMCG
15,CAMS,4762.0,0.22,1.13,23.39,24.79,59578.0,2858.0,254714.0,-3.47,47.0,X-SC,1.91,86.0,0.05,1.80,24.54,X40N,NTT,MISC
8,AWL,485.0,-0.56,-21.51,95.56,53.50,226324.0,-64911.0,236840.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.20,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.99,1.20,29.41,30.97,52603.0,2124.0,178862.0,-22.64,47.0,X-MC,2.17,75.0,0.04,1.26,13.91,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.01,-30.17,104.56,42.84,47462.0,-19614.0,45392.0,-54.27,37.0,X-SC,37.43,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.07,-1.60,-9.60,39.45,26.07,58394.0,-15720.0,148020.0,-33.39,27.0,X-MC,10.52,55.0,-0.27,1.05,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,-0.53,-5.64,41.75,33.76,79114.0,-11325.0,189495.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.23,-20.46,86.65,48.45,138184.0,-41032.0,159474.0,-26.10,14.0,X-MC,8.84,64.0,-0.30,1.13,0.23,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.95,-37.18,115.22,35.20,93137.0,-47836.0,80834.0,5.99,24.0,X-SC,14.21,92.0,-0.51,0.57,0.95,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.01,-30.17,104.56,42.84,47462.0,-19614.0,45392.0,-54.27,37.0,X-SC,37.43,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.13,-47.46,189.87,52.31,144818.0,-68888.0,76272.0,-44.05,35.0,X-SC,5.01,91.0,-0.48,0.54,2.09,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.95,-37.18,115.22,35.20,93137.0,-47836.0,80834.0,5.99,24.0,X-SC,14.21,92.0,-0.51,0.57,0.95,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.28,-9.97,35.04,21.58,44210.0,-13968.0,126171.0,-54.61,28.0,X-MC,5.88,56.0,-0.32,0.89,11.70,X40N,ATH,IT
52,MEDANTA,1486.00,-0.82,1.41,22.35,24.08,28773.0,1787.0,128737.0,-6.77,41.0,X-SC,5.81,89.0,0.06,0.91,21.30,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.07,-14.80,40.00,19.28,117902.0,-51200.0,294756.0,-25.89,58.0,X-LC,6.92,1.0,-0.43,2.08,8.97,X40,ATH,IT
41,INFY,2275.00,1.24,7.58,43.96,54.87,147971.0,23716.0,336604.0,-16.37,64.0,X-LC,2.72,2.0,0.16,2.38,16.49,X40,BTT,IT
76,TMPV,600.00,-1.63,-15.68,68.61,42.18,159918.0,-43328.0,233082.0,-24.30,31.0,X-LC,3.15,3.0,-0.27,1.65,0.96,XY24,NTT,AUTO
82,VBL,671.64,-0.87,-3.61,40.75,35.67,124065.0,-11388.0,304454.0,-14.99,58.0,X-LC,5.40,4.0,-0.09,2.15,9.75,X40N,ATH,FMCG
43,ITC,452.00,-0.17,-2.21,12.93,10.44,25307.0,-4416.0,195722.0,-41.56,40.0,X-LC,1.52,5.0,-0.17,1.38,3.23,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.24,-37.38,112.20,32.89,54249.0,-28856.0,48350.0,-697.12,60.0,L-MC,6.00,259.0,-0.53,0.34,33.15,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.54,-15.36,114.63,81.67,91022.0,-14405.0,79405.0,7138.89,43.0,L-SC,18.74,271.0,-0.16,0.56,53.77,XR,NTT,CERAMICS
51,MASFIN,398.61,0.47,-2.10,24.66,22.05,23655.0,-2055.0,95925.0,-15.54,60.0,H-SC,6.74,164.0,-0.09,0.68,39.43,XR,ATH,FINANCE
13,BSOFT,831.70,4.10,-23.57,97.30,50.79,104593.0,-33158.0,107495.0,-1.82,71.0,H-SC,5.89,151.0,-0.32,0.76,25.16,XR,ATH,IT
70,SURYODAY,216.00,-1.03,-19.79,53.40,23.03,76696.0,-35445.0,143626.0,55.70,48.0,H-SC,9.98,167.0,-0.46,1.02,42.51,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.27,-8.90,9.79,0.02,15569.0,-15529.0,159030.0,142.75,66.0,M-SC,14.25,234.0,-1.00,1.12,38.33,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.25,3.01,17.41,20.95,45152.0,7577.0,259345.0,2.04,78.0,X-LC,17.33,31.0,0.17,1.83,39.72,X40,ATH,PAINTS
13,BSOFT,831.70,4.10,-23.57,97.30,50.79,104593.0,-33158.0,107495.0,-1.82,71.0,H-SC,5.89,151.0,-0.32,0.76,25.16,XR,ATH,IT
50,LTIM,7201.88,-0.09,12.59,16.94,31.66,46946.0,30981.0,277132.0,8.32,69.0,H-LC,14.93,43.0,0.66,1.96,53.77,X200,ATH,IT
40,INDUSINDBK,1800.00,-0.24,-37.38,112.20,32.89,54249.0,-28856.0,48350.0,-697.12,60.0,L-MC,6.00,259.0,-0.53,0.34,33.15,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.03
1,25,44.82
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.40
MC    28.89
LC    25.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.56
X40      21.31
X40N     11.90
XR        9.71
XY25      9.08
AR        9.06
OX40N     7.57
X200      1.96
SR        1.00
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    22.45
X-LC    20.29
M-SC    11.91
X-SC     8.20
H-MC     4.74
H-LC     3.13
L-SC     1.38
M-MC     1.36
M-LC     1.25
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,-6.20,40.54
IT,14.10,-13.28,71.75
FINANCE,9.29,-17.91,67.74
MISC,7.45,-23.13,73.57
PAINTS,6.02,-8.16,24.22
ELECTRICAL,6.00,-10.50,50.98
INSURANCE,4.37,-2.14,38.14
PHARMA,3.89,-1.46,33.92
AUTO,3.35,-21.62,73.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3134710.0,21
AR,1290646.0,10
XR,1288184.0,13
X40,1020738.0,14
X40N,765665.0,9
OX40N,718438.0,10
XY25,357026.0,6
SR,278482.0,2
MH,72556.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3592843.0,25
M-SC,1373946.0,15
X-MC,1304202.0,16
X-LC,891329.0,11
X-SC,759464.0,8
H-MC,408823.0,3
L-SC,265938.0,3
M-LC,114853.0,1
H-LC,114357.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253946.0      6
           AR         884255.0      5
M-SC       XY24       781705.0      6
H-SC       XR         771267.0      7
X-MC       X40        460987.0      7
           XY24       387455.0      3
X-LC       X40        380749.0      5
H-SC       OX40N      332337.0      4
M-SC       OX40N      299430.0      5
X-SC       X40N       297585.0      3
           XY24       282877.0      3
H-SC       SR         278482.0      2
X-MC       X40N       269263.0      4
X-LC       XY24       234621.0      2
H-MC       AR         214717.0      2
X-LC       X40N       198817.0      2
H-MC       XY24       194106.0      1
X-MC       XY25       186497.0      2
L-SC       XR         179267.0      2
X-SC       X40        179002.0      2
M-SC       XR         168548.0      2
           AR         124263.0      2
M-LC       XR         114853.0      1
L-SC       OX40N       86671.0      1
X-LC       XY25        77142.0      2
H-SC       MH          72556.0      1
H-LC       AR          67411.0      1
L-MC       XR          54249.0      1
M-MC       XY25        52469.0      1
H-LC       X200        46946.0      1
L-LC       XY25        40918.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
